In [1]:
import os, tarfile, math
import pyBigWig
import pybedtools
import zipfile, gzip
from gtfparse import read_gtf
import pandas as pd
pd.set_option('display.max_columns', None)
from Bio import SeqIO
import pandas as pd
import numpy as np
import re 
from collections import Counter
from itertools import product
from collections import Counter

INFO:numexpr.utils:Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
gtf_file_path = "/data/projects/Resources/Gencode_genome_annotation/gencode.v40.annotation.gtf"

In [3]:
def parse_attributes(attribute_string):
    # Split each attribute into key and value, then strip quotes and spaces
    return dict(item.strip().replace('"', '').split(' ') for item in attribute_string if item)

In [4]:
column_names = [
    "seqname", "source", "feature", "start", "end",
    "score", "strand", "frame", "attribute"
]

# Read the GTF file
gtf_df = pd.read_csv(gtf_file_path, sep="\t", comment='#', header=None, names=column_names)

In [5]:
# Step 1: Split the 'attribute' string into a list of strings for each key-value pair
attributes_list = gtf_df['attribute'].str.split(';')


# Apply the function to each row's attribute list
attributes_dicts = attributes_list.apply(parse_attributes)

# Step 3: Convert the list of dictionaries into a DataFrame
attributes_df = pd.DataFrame(list(attributes_dicts))

# Step 4: Combine the new attributes DataFrame with the original gtf_df
# This step assumes that the indexes are aligned and can be directly concatenated
combined_df = pd.concat([gtf_df, attributes_df], axis=1)

# Optionally, you can drop the original 'attribute' column if it's no longer needed
combined_df = combined_df.drop('attribute', axis=1)

In [6]:
combined_df

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_type,gene_name,level,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
0,chr1,HAVANA,gene,11869,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,HAVANA,transcript,11869,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,NaN,NaN,NaN,NaN,NaN
2,chr1,HAVANA,exon,11869,12227,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,1,ENSE00002234944.1,NaN,NaN,NaN
3,chr1,HAVANA,exon,12613,12721,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,2,ENSE00003582793.1,NaN,NaN,NaN
4,chr1,HAVANA,exon,13221,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,3,ENSE00002312635.1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283857,chrM,ENSEMBL,transcript,15888,15953,.,+,.,ENSG00000210195.2,Mt_tRNA,MT-TT,3,HGNC:7499,NaN,ENST00000387460.2,Mt_tRNA,MT-TT-201,NA,Ensembl_canonical,NaN,NaN,NaN,NaN,NaN,NaN
3283858,chrM,ENSEMBL,exon,15888,15953,.,+,.,ENSG00000210195.2,Mt_tRNA,MT-TT,3,HGNC:7499,NaN,ENST00000387460.2,Mt_tRNA,MT-TT-201,NA,Ensembl_canonical,NaN,1,ENSE00001544475.2,NaN,NaN,NaN
3283859,chrM,ENSEMBL,gene,15956,16023,.,-,.,ENSG00000210196.2,Mt_tRNA,MT-TP,3,HGNC:7494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3283860,chrM,ENSEMBL,transcript,15956,16023,.,-,.,ENSG00000210196.2,Mt_tRNA,MT-TP,3,HGNC:7494,NaN,ENST00000387461.2,Mt_tRNA,MT-TP-201,NA,Ensembl_canonical,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Filter rows where 'feature' is 'transcript' and 'seqname' is not 'chrM'
gtf_transcript = combined_df[(combined_df["feature"] == "transcript") & (combined_df["seqname"] != "chrM")]

# Display the resulting DataFrame
gtf_transcript

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_type,gene_name,level,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,ont,protein_id,ccdsid
1,chr1,HAVANA,transcript,11869,14409,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,1,basic,OTTHUMT00000362751.1,NaN,NaN,NaN,NaN,NaN
5,chr1,HAVANA,transcript,12010,13670,.,+,.,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,2,HGNC:37102,OTTHUMG00000000961.2,ENST00000450305.2,transcribed_unprocessed_pseudogene,DDX11L1-201,NA,Ensembl_canonical,OTTHUMT00000002844.2,NaN,NaN,PGO:0000019,NaN,NaN
13,chr1,HAVANA,transcript,14404,29570,.,-,.,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,2,HGNC:38034,OTTHUMG00000000958.1,ENST00000488147.1,unprocessed_pseudogene,WASH7P-201,NA,Ensembl_canonical,OTTHUMT00000002839.1,NaN,NaN,PGO:0000005,NaN,NaN
26,chr1,ENSEMBL,transcript,17369,17436,.,-,.,ENSG00000278267.1,miRNA,MIR6859-1,3,HGNC:50039,NaN,ENST00000619216.1,miRNA,MIR6859-1-201,NA,Ensembl_canonical,NaN,NaN,NaN,NaN,NaN,NaN
29,chr1,HAVANA,transcript,29554,31097,.,+,.,ENSG00000243485.5,lncRNA,MIR1302-2HG,2,HGNC:52482,OTTHUMG00000000959.2,ENST00000473358.1,lncRNA,MIR1302-2HG-202,5,Ensembl_canonical,OTTHUMT00000002840.1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283698,chrY,HAVANA,transcript,57209306,57210051,.,+,.,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,2,HGNC:31685,OTTHUMG00000022677.5,ENST00000483079.6_PAR_Y,retained_intron,WASH6P-210,1,PAR,OTTHUMT00000058833.1,NaN,NaN,NaN,NaN,NaN
3283701,chrY,HAVANA,transcript,57209887,57212186,.,+,.,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,2,HGNC:31685,OTTHUMG00000022677.5,ENST00000496301.6_PAR_Y,retained_intron,WASH6P-215,2,PAR,OTTHUMT00000058827.1,NaN,NaN,NaN,NaN,NaN
3283704,chrY,HAVANA,transcript,57210344,57212074,.,+,.,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,2,HGNC:31685,OTTHUMG00000022677.5,ENST00000483286.6_PAR_Y,retained_intron,WASH6P-211,1,PAR,OTTHUMT00000058834.1,NaN,NaN,NaN,NaN,NaN
3283708,chrY,HAVANA,transcript,57210591,57212074,.,+,.,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,2,HGNC:31685,OTTHUMG00000022677.5,ENST00000464205.6_PAR_Y,processed_transcript,WASH6P-205,2,PAR,OTTHUMT00000058835.1,NaN,NaN,NaN,NaN,NaN


In [8]:
gtf_transcript.groupby('transcript_type').size()

transcript_type
IG_C_gene                                23
IG_C_pseudogene                           9
IG_D_gene                                37
IG_J_gene                                18
IG_J_pseudogene                           3
IG_V_gene                               145
IG_V_pseudogene                         187
IG_pseudogene                             1
TEC                                    1147
TR_C_gene                                 6
TR_D_gene                                 4
TR_J_gene                                79
TR_J_pseudogene                           4
TR_V_gene                               106
TR_V_pseudogene                          33
lncRNA                                51324
miRNA                                  1879
misc_RNA                               2212
non_stop_decay                           99
nonsense_mediated_decay               20254
polymorphic_pseudogene                   71
processed_pseudogene                  10156
processed_transc

In [9]:
def extract_tss_regions_from_df(tss_df):
    """
    Extracts TSS (+50 and -50 bp) regions from the GTF DataFrame.
    :param df: A pandas DataFrame containing GTF data.
    :return: A DataFrame with TSS regions.
    """

    # Calculate TSS based on strand
    tss_df['TSS'] = tss_df.apply(lambda x: x['start'] if x['strand'] == '+' else x['end'], axis=1)

    # # Adjust TSS based on strand for +50 and -50 bp
    # tss_df['start_adj'] = tss_df.apply(lambda x: x['TSS'] - 45 if x['strand'] == '+' else x['TSS'], axis=1)
    # tss_df['end_adj'] = tss_df.apply(lambda x: x['TSS'] + 45 if x['strand'] == '+' else x['TSS'] + 90, axis=1)
    
     # Adjust TSS based on strand for +50 and -50 bp
    tss_df['start_adj'] = tss_df.apply(lambda x: x['TSS'] - 45, axis=1)
    tss_df['end_adj'] = tss_df.apply(lambda x: x['TSS'] + 45, axis=1)

    # Select and rename relevant columns
    tss_regions = tss_df[['seqname', 'TSS','start_adj', 'end_adj', 'strand', 'gene_id', 'gene_type', 'gene_name', 'hgnc_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name']]
    tss_regions.columns = ['Chromosome', 'TSS', '45BP_Start', '45BP_End', 'Strand', 'GeneID', 'gene_type', 'gene_name', 'hgnc_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name']

    return tss_regions

In [10]:
tss_regions_df = extract_tss_regions_from_df(gtf_transcript)

/tmp/ipykernel_4146629/1735076899.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tss_df['TSS'] = tss_df.apply(lambda x: x['start'] if x['strand'] == '+' else x['end'], axis=1)
/tmp/ipykernel_4146629/1735076899.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tss_df['start_adj'] = tss_df.apply(lambda x: x['TSS'] - 45, axis=1)
/tmp/ipykernel_4146629/1735076899.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [11]:
tss_regions_df

,Chromosome,TSS,45BP_Start,45BP_End,Strand,GeneID,gene_type,gene_name,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name
1,chr1,11869,11824,11914,+,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202
5,chr1,12010,11965,12055,+,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000450305.2,transcribed_unprocessed_pseudogene,DDX11L1-201
13,chr1,29570,29525,29615,-,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,HGNC:38034,OTTHUMG00000000958.1,ENST00000488147.1,unprocessed_pseudogene,WASH7P-201
26,chr1,17436,17391,17481,-,ENSG00000278267.1,miRNA,MIR6859-1,HGNC:50039,NaN,ENST00000619216.1,miRNA,MIR6859-1-201
29,chr1,29554,29509,29599,+,ENSG00000243485.5,lncRNA,MIR1302-2HG,HGNC:52482,OTTHUMG00000000959.2,ENST00000473358.1,lncRNA,MIR1302-2HG-202
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283698,chrY,57209306,57209261,57209351,+,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483079.6_PAR_Y,retained_intron,WASH6P-210
3283701,chrY,57209887,57209842,57209932,+,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000496301.6_PAR_Y,retained_intron,WASH6P-215
3283704,chrY,57210344,57210299,57210389,+,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483286.6_PAR_Y,retained_intron,WASH6P-211
3283708,chrY,57210591,57210546,57210636,+,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000464205.6_PAR_Y,processed_transcript,WASH6P-205


In [14]:
tss_regions_df[tss_regions_df['TSS']==1000096]

,Chromosome,TSS,45BP_Start,45BP_End,Strand,GeneID,gene_type,gene_name,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name
1576,chr1,1000096,1000051,1000141,-,ENSG00000188290.11,protein_coding,HES4,HGNC:24149,OTTHUMG00000040758.2,ENST00000481869.1,retained_intron,HES4-203


In [16]:
# Define a custom aggregation function
def custom_agg(series):
    # If all values are the same, return any one of them; otherwise, return the list of unique values
    if series.nunique() == 1:
        return series.iloc[0]
    else:
        return list(series.unique())

In [18]:
# Group by the columns 'Chromosome', 'Start', 'End', 'Strand'
# and aggregate other columns
unique_tss_df = tss_regions_df.groupby(['Chromosome', '45BP_Start', '45BP_End', 'Strand']).agg(custom_agg).reset_index()

In [19]:
unique_tss_df

,Chromosome,45BP_Start,45BP_End,Strand,TSS,GeneID,gene_type,gene_name,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name
0,chr1,11824,11914,+,11869,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202
1,chr1,11965,12055,+,12010,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000450305.2,transcribed_unprocessed_pseudogene,DDX11L1-201
2,chr1,17391,17481,-,17436,ENSG00000278267.1,miRNA,MIR6859-1,HGNC:50039,[nan],ENST00000619216.1,miRNA,MIR6859-1-201
3,chr1,29509,29599,+,29554,ENSG00000243485.5,lncRNA,MIR1302-2HG,HGNC:52482,OTTHUMG00000000959.2,ENST00000473358.1,lncRNA,MIR1302-2HG-202
4,chr1,29525,29615,-,29570,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,HGNC:38034,OTTHUMG00000000958.1,ENST00000488147.1,unprocessed_pseudogene,WASH7P-201
...,...,...,...,...,...,...,...,...,...,...,...,...,...
215293,chrY,57209261,57209351,+,57209306,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483079.6_PAR_Y,retained_intron,WASH6P-210
215294,chrY,57209842,57209932,+,57209887,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000496301.6_PAR_Y,retained_intron,WASH6P-215
215295,chrY,57210299,57210389,+,57210344,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483286.6_PAR_Y,retained_intron,WASH6P-211
215296,chrY,57210546,57210636,+,57210591,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000464205.6_PAR_Y,processed_transcript,WASH6P-205


In [22]:
def read_genome(fasta_file):
    """
    Reads a genome fasta file and returns a dictionary of chromosomes and sequences.
    :param fasta_file: Path to the genome fasta file.
    :return: A dictionary with chromosome names as keys and sequences as values.
    """
    genome = {}
    for seq_record in SeqIO.parse(fasta_file, "fasta"):
        genome[seq_record.id] = seq_record.seq
    return genome

def fetch_sequences_for_regions(df, genome):
    """
    Fetches sequences for regions specified in the DataFrame from the genome.
    :param df: DataFrame with the regions (must contain 'Chromosome', 'Start', 'End', and 'Strand' columns).
    :param genome: Dictionary of genome sequences.
    :return: List of sequences corresponding to the regions.
    """
    sequences = []
    for index, row in df.iterrows():
        # Extracting the sequence
        seq = genome[row['Chromosome']][row['45BP_Start']:row['45BP_End']]
        
        # Reverse complement if the gene is on the negative strand
        if row['Strand'] == '-':
            seq = seq.reverse_complement()
        seq= seq.upper()
        
        sequences.append(str(seq))  # Converts Seq object to string
    return sequences

In [23]:
# Paths
genome_path = "/data/projects/Resources/Gencode_genome_annotation/GRCh38.primary_assembly.genome.fa"
# Load the genome and TSS regions
genome = read_genome(genome_path)
# Fetch sequences
sequences = fetch_sequences_for_regions(unique_tss_df, genome)

In [24]:
unique_tss_df['Sequence'] = sequences  # Add sequences to the DataFrame
unique_tss_df

,Chromosome,45BP_Start,45BP_End,Strand,TSS,GeneID,gene_type,gene_name,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name,Sequence
0,chr1,11824,11914,+,11869,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,AACGAGATTGCCAGCACCGGGTATCATTCACCATTTTTCTTTTCGT...
1,chr1,11965,12055,+,12010,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000450305.2,transcribed_unprocessed_pseudogene,DDX11L1-201,TTTGAGAGGTCACAGGGTCTTGATGCTGTGGTCTTCATCTGCAGGT...
2,chr1,17391,17481,-,17436,ENSG00000278267.1,miRNA,MIR6859-1,HGNC:50039,[nan],ENST00000619216.1,miRNA,MIR6859-1-201,CTCGGGAGGCCTGGGACCCCACAGTGCACGCTGTGCCCCTGATGAT...
3,chr1,29509,29599,+,29554,ENSG00000243485.5,lncRNA,MIR1302-2HG,HGNC:52482,OTTHUMG00000000959.2,ENST00000473358.1,lncRNA,MIR1302-2HG-202,CGCGGGCCTGGGCACGGAACTCACGCTCACTCCGAGCTCCCGACGT...
4,chr1,29525,29615,-,29570,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,HGNC:38034,OTTHUMG00000000958.1,ENST00000488147.1,unprocessed_pseudogene,WASH7P-201,GCAGAAAGCACGGGTAGGGGCGGCCTGACGCTCGGAAGACAACGCA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215293,chrY,57209261,57209351,+,57209306,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483079.6_PAR_Y,retained_intron,WASH6P-210,ACAAAGACCCATGTGATGCTGGGGGCAGAGACAGAGGAGAAGCTGT...
215294,chrY,57209842,57209932,+,57209887,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000496301.6_PAR_Y,retained_intron,WASH6P-215,CACCACCCCCACCGCCCCCACCACCACCCCCAGCTCCTGAGGTGCT...
215295,chrY,57210299,57210389,+,57210344,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483286.6_PAR_Y,retained_intron,WASH6P-211,TAACCCTAGCATCCAGAAGTGGCACAAAACCCCTCTGCTGGCTCAT...
215296,chrY,57210546,57210636,+,57210591,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000464205.6_PAR_Y,processed_transcript,WASH6P-205,TCCCTCCCAGCAGGCTCTTGGACACAGTAAGCTTCCCCAGCCCTGC...


## TATA Box pattern matching

In [25]:
# Your existing function
def find_tata_box(sequence):
    # tata_pattern_plus = r"TAT[AT][AT][AG]"  # Simplified TATA box pattern
    # tata_pattern_minus= r"[TC][AT][AT]ATA"
    tata_pattern = r"TAT[AT][AT][AG]|[TC][AT][AT]ATA"
    for match in re.finditer(tata_pattern, sequence):
        yield match.start()

In [26]:
# Assuming 'unique_tss_df' is your DataFrame with the sequences

# Apply the function to each sequence and store the results in a new column
unique_tss_df['TATA_Box_Positions'] = unique_tss_df['Sequence'].apply(lambda seq: list(find_tata_box(seq)))

In [27]:
unique_tss_df

,Chromosome,45BP_Start,45BP_End,Strand,TSS,GeneID,gene_type,gene_name,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name,Sequence,TATA_Box_Positions
0,chr1,11824,11914,+,11869,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000456328.2,processed_transcript,DDX11L1-202,AACGAGATTGCCAGCACCGGGTATCATTCACCATTTTTCTTTTCGT...,[]
1,chr1,11965,12055,+,12010,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,HGNC:37102,OTTHUMG00000000961.2,ENST00000450305.2,transcribed_unprocessed_pseudogene,DDX11L1-201,TTTGAGAGGTCACAGGGTCTTGATGCTGTGGTCTTCATCTGCAGGT...,[]
2,chr1,17391,17481,-,17436,ENSG00000278267.1,miRNA,MIR6859-1,HGNC:50039,[nan],ENST00000619216.1,miRNA,MIR6859-1-201,CTCGGGAGGCCTGGGACCCCACAGTGCACGCTGTGCCCCTGATGAT...,[]
3,chr1,29509,29599,+,29554,ENSG00000243485.5,lncRNA,MIR1302-2HG,HGNC:52482,OTTHUMG00000000959.2,ENST00000473358.1,lncRNA,MIR1302-2HG-202,CGCGGGCCTGGGCACGGAACTCACGCTCACTCCGAGCTCCCGACGT...,[]
4,chr1,29525,29615,-,29570,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,HGNC:38034,OTTHUMG00000000958.1,ENST00000488147.1,unprocessed_pseudogene,WASH7P-201,GCAGAAAGCACGGGTAGGGGCGGCCTGACGCTCGGAAGACAACGCA...,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215293,chrY,57209261,57209351,+,57209306,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483079.6_PAR_Y,retained_intron,WASH6P-210,ACAAAGACCCATGTGATGCTGGGGGCAGAGACAGAGGAGAAGCTGT...,[]
215294,chrY,57209842,57209932,+,57209887,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000496301.6_PAR_Y,retained_intron,WASH6P-215,CACCACCCCCACCGCCCCCACCACCACCCCCAGCTCCTGAGGTGCT...,[]
215295,chrY,57210299,57210389,+,57210344,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000483286.6_PAR_Y,retained_intron,WASH6P-211,TAACCCTAGCATCCAGAAGTGGCACAAAACCCCTCTGCTGGCTCAT...,[]
215296,chrY,57210546,57210636,+,57210591,ENSG00000182484.15_PAR_Y,protein_coding,WASH6P,HGNC:31685,OTTHUMG00000022677.5,ENST00000464205.6_PAR_Y,processed_transcript,WASH6P-205,TCCCTCCCAGCAGGCTCTTGGACACAGTAAGCTTCCCCAGCCCTGC...,[]


In [30]:
# Create the three dataframes based on the conditions
# 1. TATA Core Promoter DataFrame
tata_core_df = unique_tss_df[unique_tss_df['TATA_Box_Positions'].apply(
    lambda positions: any(10 <= pos <= 20 for pos in positions))]

# 2. Sequences containing TATA Box but not functioning as TATA core promoter
tata_non_functional_df = unique_tss_df[unique_tss_df['TATA_Box_Positions'].apply(
    lambda positions: any(pos not in range(10, 20) for pos in positions) if positions else False)]

# 3. DataFrame without TATA Box
no_tata_df = unique_tss_df[unique_tss_df['TATA_Box_Positions'].apply(len) == 0]

In [31]:
tata_core_df

,Chromosome,45BP_Start,45BP_End,Strand,TSS,GeneID,gene_type,gene_name,hgnc_id,havana_gene,transcript_id,transcript_type,transcript_name,Sequence,TATA_Box_Positions
31,chr1,182651,182741,+,182696,ENSG00000279928.2,unprocessed_pseudogene,DDX11L17,HGNC:55080,OTTHUMG00000191962.1,ENST00000624431.2,unprocessed_pseudogene,DDX11L17-201,GTTGTCTGCATGTAACTTAATACCACAACCAGGCATAGGGGAAAGA...,[16]
42,chr1,359636,359726,-,359681,ENSG00000228463.10,transcribed_processed_pseudogene,ENSG00000228463,[nan],OTTHUMG00000002552.3,ENST00000441866.2,processed_transcript,ENST00000441866,TCTTTCTTCTTTATTAAAAAAAAGGAAAAATCCAAACACAAGATTA...,"[11, 43]"
58,chr1,498931,499021,-,498976,ENSG00000237094.12,transcribed_unprocessed_pseudogene,ENSG00000237094,[nan],OTTHUMG00000002857.7,"[ENST00000616311.5, ENST00000616947.2]",processed_transcript,"[ENST00000616311, ENST00000616947]",GTATGTTATGAAGGTTATATGTTAGGAAGGGTCCCAGGAGGTAGAC...,[15]
72,chr1,588408,588498,-,588453,ENSG00000230021.10,transcribed_processed_pseudogene,ENSG00000230021,[nan],OTTHUMG00000191652.4,ENST00000417636.2,processed_transcript,ENST00000417636,TCTTTCTTCTTTATTAAAAAAAAGGAAAAATCCAAACACAAGATTA...,"[11, 43]"
121,chr1,810116,810206,-,810161,ENSG00000230092.7,transcribed_unprocessed_pseudogene,ENSG00000230092,[nan],OTTHUMG00000002403.3,ENST00000590817.3,transcribed_unprocessed_pseudogene,ENST00000590817,GAAAAAATATTTTAATAGAATATGTCTTTATTTCAGTATCTTAAAT...,"[11, 41]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215249,chrY,25995652,25995742,-,25995697,ENSG00000231716.1,unprocessed_pseudogene,CDY23P,HGNC:23867,OTTHUMG00000045273.1,ENST00000451661.1,unprocessed_pseudogene,CDY23P-201,CGAACAGCAGTACAAATAAACAAAGATTAAGGTAAAGATAAATATC...,"[12, 38]"
215253,chrY,26123629,26123719,+,26123674,ENSG00000215507.10,transcribed_processed_pseudogene,RBMY2DP,HGNC:23259,OTTHUMG00000045283.2,ENST00000448881.5,transcribed_processed_pseudogene,RBMY2DP-202,CTATTTTTATCTAATATAGATTTTTTTTAACCATTTACAGCACAAT...,[10]
215258,chrY,26247339,26247429,+,26247384,ENSG00000251796.1,snoRNA,SNORA70,[nan],[nan],ENST00000515987.1,snoRNA,SNORA70.11-201,TTCTCTTCCTAATACTATTTGTTTACACCTATAGTTTTTCTCTAGA...,"[8, 15]"
215270,chrY,57016051,57016141,-,57016096,ENSG00000237801.6_PAR_Y,processed_pseudogene,AMD1P2,HGNC:460,OTTHUMG00000022673.2,ENST00000412936.6_PAR_Y,processed_pseudogene,AMD1P2-201,TAGTTGATTTTCTGTGGCTATTTGTTGTTTGCTAGTCTCATGGTGA...,[18]


In [32]:
tata_core_df.to_csv("/data/private/pdutta_new/non_coding_regions/TATA_NonTATA/Pattern/TATA_raw_positive.tsv", sep="\t", index=False)

In [114]:
# Your existing function
def find_tata_box_stringent(sequence):
    # tata_pattern_plus = r"TAT[AT][AT][AG]"  # Simplified TATA box pattern
    # tata_pattern_minus= r"[TC][AT][AT]ATA"
    tata_pattern = r"TATA[AT][AT][AG]"
    for match in re.finditer(tata_pattern, sequence):
        yield match.start()

In [33]:
# Initialize a list to hold results
results = []
#[f'chr{i}' for i in range(1, 23)] +
# Iterate through specified chromosomes only
for chrom in [f'chr{i}' for i in range(1, 23)]+ ['chrX', 'chrY']:
    sequence = genome[chrom]  # Fetch the sequence for the chromosome
    length = len(sequence)
    print(chrom)
    #print(sequence[0:10])

    # Break down the chromosome into 100 bp segments and search
    for i in range(0, length - 90 + 1, 50):  # 50 bp step for some overlap; adjust as needed
        segment = str(sequence[i:i+90]).upper()  # Ensure segment is a string and uppercase
        #print(segment)
        # Check if segment is a proper string before processing
        if isinstance(segment, str):
            for pos in find_tata_box(segment):
                if 10 <= pos <= 20:
                    results.append({
                        'Chromosome': chrom,
                        'Start': i,
                        'End': i + 90,
                        'TATA_Box_Position': pos,
                        'Sequence': segment
                    })
        else:
            print(f"Non-string segment detected at {chrom}:{i}-{i+90}")

# Convert results into a DataFrame
tata_box_df = pd.DataFrame(results)

chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
chrY


In [34]:
tata_box_df

,Chromosome,Start,End,TATA_Box_Position,Sequence
0,chr1,11500,11590,12,TCAGTAGACTCCTAAATATGGGATTCCTGGGTTTAAAAGTAAAAAA...
1,chr1,11900,11990,17,TTCTTTCTGTTCATGTGTATTTGCTGTCTCTTAGCCCAGACTTCCC...
2,chr1,20650,20740,16,GGAAACTTCTGTTAACTATAAGCTCAGTAGGGGCTAAAAGCATGTT...
3,chr1,26200,26290,18,ACTTATTTTTCCTCCCATCAAATAACTAAAGCATGGCCAGCTGATG...
4,chr1,28250,28340,15,TAGTATCTATTTTTACAAATAAGAAAACCCAGGCACAAAGGGGTTG...
...,...,...,...,...,...
4202322,chrY,57201200,57201290,16,TATAGGAATGAGAATGCTTATAAAGCAGACGTCATTTTATGTACCA...
4202323,chrY,57204300,57204390,16,TCTAACAAAATACATACAAATATGCATTCAGAAAACTTTAGATCAC...
4202324,chrY,57204350,57204440,15,GAGAAGAATCAAAAATATTAAATCAAATGCAGATACTCCTTGTTTA...
4202325,chrY,57204500,57204590,17,AACCCAAAAAACCCGAGTAAATACATTCAATTGATTTTGTCAAAGG...


In [35]:
# Convert DataFrames to BED format for pybedtools
tata_box_bed = pybedtools.BedTool.from_dataframe(tata_box_df[['Chromosome', 'Start', 'End']])
tata_core_bed = pybedtools.BedTool.from_dataframe(tata_core_df[['Chromosome', '45BP_Start', '45BP_End']])
# Use the intersect function with the -v option to get non-intersecting regions
non_intersecting = tata_box_bed.intersect(tata_core_bed, v=True)

In [36]:
# Convert back to DataFrame
non_intersecting_df = non_intersecting.to_dataframe(names=['Chromosome', 'Start', 'End'])

# You might want to merge back additional information from tata_box_df if needed
tata_box_df_filtered = pd.merge(non_intersecting_df, tata_box_df, on=['Chromosome', 'Start', 'End'])

In [37]:
tata_box_df_filtered

,Chromosome,Start,End,TATA_Box_Position,Sequence
0,chr1,11500,11590,12,TCAGTAGACTCCTAAATATGGGATTCCTGGGTTTAAAAGTAAAAAA...
1,chr1,11900,11990,17,TTCTTTCTGTTCATGTGTATTTGCTGTCTCTTAGCCCAGACTTCCC...
2,chr1,20650,20740,16,GGAAACTTCTGTTAACTATAAGCTCAGTAGGGGCTAAAAGCATGTT...
3,chr1,26200,26290,18,ACTTATTTTTCCTCCCATCAAATAACTAAAGCATGGCCAGCTGATG...
4,chr1,28250,28340,15,TAGTATCTATTTTTACAAATAAGAAAACCCAGGCACAAAGGGGTTG...
...,...,...,...,...,...
4429129,chrY,57201200,57201290,16,TATAGGAATGAGAATGCTTATAAAGCAGACGTCATTTTATGTACCA...
4429130,chrY,57204300,57204390,16,TCTAACAAAATACATACAAATATGCATTCAGAAAACTTTAGATCAC...
4429131,chrY,57204350,57204440,15,GAGAAGAATCAAAAATATTAAATCAAATGCAGATACTCCTTGTTTA...
4429132,chrY,57204500,57204590,17,AACCCAAAAAACCCGAGTAAATACATTCAATTGATTTTGTCAAAGG...


In [38]:
tata_box_df_filtered.to_csv("/data/private/pdutta_new/non_coding_regions/TATA_NonTATA/Pattern/TATA_raw_negative_all.tsv", sep="\t", index=False)

In [39]:
tata_box_df_filtered_sample = tata_box_df_filtered.sample(n=8500, random_state=1).reset_index(drop=True)

In [40]:
tata_box_df_filtered_sample

,Chromosome,Start,End,TATA_Box_Position,Sequence
0,chr7,113360450,113360540,13,AAATATGTATATATATTTAGAGTGTTTCTTTACTCTTGCTTGTGTT...
1,chr1,83076800,83076890,19,CCATAAGTTTCTATTCCAATATATAACTTTTTTTCTTTGCAGAAAA...
2,chr3,68511800,68511890,17,TTTAAAATTATAGTAGATAAATATAAGTAATACACAATATTTAATT...
3,chr8,50220000,50220090,10,AACTTTAAAATATATAAAGTCAAAAATGAAAGAACTACAAGGAGAA...
4,chr8,120829250,120829340,12,GACAATTACTACTATTAACAATAATAACATATAATAGCTATAAACA...
...,...,...,...,...,...
8495,chr16,58565400,58565490,19,CACCATGCTCAGCCTTCACTATATATTTTCTATGAACAAAAACATT...
8496,chr9,67640800,67640890,10,TAGTATGAACCTAATACACGAGACAAGTTAAAAAATCAGAGTTGGC...
8497,chr2,92852850,92852940,10,GGATTGTCTTCATATAAACTCTAGACAGAAGCATTCTCAGAAGCTT...
8498,chr4,32808500,32808590,11,TTTGGGCTGCATATTAAAACTTACAGTACAAGTTGACTAATACTTA...


In [127]:
tata_box_df_filtered_sample.to_csv("/data/private/pdutta/DNABERT_data/Core_Prom_new/TATA/raw_negative.tsv", sep="\t", index=False)

In [8]:
df_pos

,Sequence
0,GCTTTG CTTTGC TTTGCA TTGCAT TGCATG GCATGC CATG...
1,GGACGG GACGGA ACGGAG CGGAGC GGAGCG GAGCGG AGCG...
2,CTCCAC TCCACT CCACTT CACTTC ACTTCC CTTCCT TTCC...
3,TGGCCC GGCCCC GCCCCG CCCCGC CCCGCC CCGCCC CGCC...
4,TCCCTC CCCTCC CCTCCC CTCCCA TCCCAC CCCACC CCAC...
...,...
4320,GTGCGA TGCGAG GCGAGG CGAGGA GAGGAG AGGAGT GGAG...
4321,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...
4322,CGCGGC GCGGCA CGGCAC GGCACA GCACAG CACAGG ACAG...
4323,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...


In [9]:
df_pos['Label']=1

In [10]:
df_pos = df_pos.reset_index(drop=True)
df_pos

,Sequence,Label
0,GCTTTG CTTTGC TTTGCA TTGCAT TGCATG GCATGC CATG...,1
1,GGACGG GACGGA ACGGAG CGGAGC GGAGCG GAGCGG AGCG...,1
2,CTCCAC TCCACT CCACTT CACTTC ACTTCC CTTCCT TTCC...,1
3,TGGCCC GGCCCC GCCCCG CCCCGC CCCGCC CCGCCC CGCC...,1
4,TCCCTC CCCTCC CCTCCC CTCCCA TCCCAC CCCACC CCAC...,1
...,...,...
236970,GTGCGA TGCGAG GCGAGG CGAGGA GAGGAG AGGAGT GGAG...,1
236971,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...,1
236972,CGCGGC GCGGCA CGGCAC GGCACA GCACAG CACAGG ACAG...,1
236973,ATGACA TGACAG GACAGT ACAGTC CAGTCC AGTCCT GTCC...,1


In [12]:
df_pos.to_csv("/data/projects/DNABERT_data/Core_promoters/Core_promoter_regions/positive_set.tsv", sep="\t", index=False)